In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv('data/creditcard.csv')
data.drop('Time', axis=1, inplace=True)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data.drop('Class', axis=1), 
    data['Class'],
    test_size=0.2,
    random_state=42,
    stratify=data['Class'],
    shuffle=True
)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(pd.DataFrame(X_train['Amount']))
X_train['Amount'] = scaler.transform(pd.DataFrame(X_train['Amount']))
X_test['Amount'] = scaler.transform(pd.DataFrame(X_test['Amount']))

# Oversampling

## Smote

In [5]:
y_train.value_counts()

0    227451
1       394
Name: Class, dtype: int64

In [17]:
from imblearn.over_sampling import SMOTE 

sm = SMOTE({1 : 5000}, random_state=42, n_jobs=-1) # neighbor = 5
X_smote, y_smote = sm.fit_resample(X_train, y_train)

C:\Users\ghktj\anaconda3\envs\pytorch\lib\site-packages\imblearn\utils\_validation.py:587: FutureWarning: Pass sampling_strategy={1: 5000} as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


In [18]:
y_smote.value_counts()

0    227451
1      5000
Name: Class, dtype: int64

In [19]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_smote, y_smote)
train_pred = model.predict(X_smote)
test_pred = model.predict(X_test)
train_pred_prob = model.predict_proba(X_smote)[:, 1]
test_pred_prob = model.predict_proba(X_test)[:, 1]

In [20]:
from score import printScore

printScore(train_pred, test_pred, train_pred_prob, test_pred_prob, y_smote, y_test)

Train
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    227451
           1       0.97      0.82      0.89      5000

    accuracy                           1.00    232451
   macro avg       0.99      0.91      0.95    232451
weighted avg       1.00      1.00      1.00    232451

Test
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.68      0.84      0.75        98

    accuracy                           1.00     56962
   macro avg       0.84      0.92      0.87     56962
weighted avg       1.00      1.00      1.00     56962


ROC AUC Train = 0.9878176451191685
ROC AUC Test = 0.9636560343045492

Average precision socre Train = 0.9029361399207173
Average precision socre Test = 0.745749916593066
